In [26]:
import tarfile

import tqdm
import xarray as xr
import hvplot.xarray

import radolan_to_netcdf as rtn

# Download RADKLIM-YW data for May 2016

We want to look at the data for the flooding in the city of Braunsbach on 29th of May 2016.

**Note that RADKLIM data is corrected based on long-term statistics. Hence, these correction are not adjusted for individual events, like the one shown here. Like all radar rainfall estimate, and all rainfall estimates in general, the absolute values have to be treated with caution. The data is shown here only to highlight how easy it now is to access and explore RADOLAN and RADKLIM data.**

In [22]:
!curl -O "https://opendata.dwd.de/climate_environment/CDC/grids_germany/5_minutes/radolan/reproc/2017_002/bin/supplement/YW2017.002_Supplement.tar.gz"

# #importing the module
# import tarfile

# #declaring the filename
# name_of_file= 'C:/Users/MSA/Downloads/YW2017.002_202107.tar'

# #opening the file in write mode
# file= tarfile.open(name_of_file,"w")

# #Adding other files to the tar file
# file.add('C:/Users/MSA/Downloads/07/')

# #closing the file
# file.close()

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 6082k  100 6082k    0     0  2022k      0  0:00:03  0:00:03 --:--:-- 2025k
100 6082k  100 6082k    0     0  2021k      0  0:00:03  0:00:03 --:--:-- 2025k


# Read data from nested tar file

Data is provided as monthly tar files, which contains dailt tar.gz files, which contain the 5-minute binary files. To avoid extracting everything first we use the nested loop-construct below and extract only the data we want on the fly.

In [32]:
fn_netcdf = './radklim-yw.nc'
rtn.create_empty_netcdf(fn=fn_netcdf, product_name='YW')

with tarfile.open('P:/Basel/CHF10017_HWS_Aubach/1_Konzept_Beratung/2_Berechnungen/netCDF2tif/YW2017.002_202107_direct.tar.gz', 'r') as tar:
    fn_list = tar.getnames()
    print(fn_list)
    fn_list.sort()
    for fn in fn_list[28:29]:
        print(f'Extracting daily file {fn}')
        f = tar.extractfile(fn)
        with tarfile.open(fileobj=f) as tar_inner:
            fn_list_inner = tar_inner.getnames()
            fn_list_inner.sort()
            # We only extract the 5-minute data from the afternoon
            for fn_inner in tqdm.tqdm(fn_list_inner[-120:-60]):
                #print(f'parsing {fn_inner}')
                data, metadata = rtn.read_in_one_bin_file(tar_inner.extractfile(fn_inner))
                rtn.append_to_netcdf(
                    fn_netcdf, 
                    data_list=[data, ], 
                    metadata_list=[metadata, ],
                )

OverflowError: Python integer -9999 out of bounds for int8

In [ ]:

with tarfile.open('YW2017.002_202107_direct.tar', 'r') as tar:
    fn_list = tar.getnames()
    print(fn_list)
    fn_list.sort()
    for fn in fn_list[28:29]:
        print(f'Extracting daily file {fn}')
        f = tar.extractfile(fn)
        with tarfile.open(fileobj=f) as tar_inner:
            fn_list_inner = tar_inner.getnames()
            fn_list_inner.sort()
            # We only extract the 5-minute data from the afternoon
            for fn_inner in tqdm.tqdm(fn_list_inner[-120:-60]):
                #print(f'parsing {fn_inner}')
                data, metadata = rtn.read_in_one_bin_file(tar_inner.extractfile(fn_inner))
                rtn.append_to_netcdf(
                    fn_netcdf, 
                    data_list=[data, ], 
                    metadata_list=[metadata, ],
                )

In [ ]:
# ds = xr.open_dataset('./data/YW_2017.002_20210715.nc')
ds = xr.open_dataset('C:/Users/MSA/Downloads/YW2017.002_2021_netcdf_supplement.tar.gz')
ds

# Plot 5-minute rainfall maps with time slider 

Note that I have not yet succeeded in adding a marker for the locations of Braunsbach in this plot. See the WIP version at the end of this notebook.

In [6]:
rainfall_map = ds.rainfall_amount.hvplot.quadmesh(
    x='longitudes', 
    y='latitudes',
    frame_width=500, 
    rasterize=True,
    tiles='OSM', 
    project=True, 
    geo=True, 
    clim=(0.1, 10), 
    cmap='rainbow', 
    clabel='rainfall amount (mm)')

rainfall_map.opts('Image', clipping_colors={'min': 'transparent', 'NaN': 'gray'}, alpha=0.5, toolbar='above')

rainfall_map

AttributeError: 'Dataset' object has no attribute 'rainfall_amount'

# Plot rainfall accumulation over the period covered by the data

Add coordinates of the village of Braunsbach as `pandas.Dataframe`, because this currently seems to be the easiest way to add a marker to the `hvplot` map.

In [6]:
import pandas as pd
import hvplot.pandas

braunsbach = pd.DataFrame(data={'x': [9.7913, ], 'y': [49.1992, ]})

In [7]:
print('First time stamp in data: ' + str(ds.rainfall_amount.time.values[0]))
print('Last time stamp in data : ' + str(ds.rainfall_amount.time.values[-1]))

First time stamp in data: 2016-05-29T13:59:59.999999936
Last time stamp in data : 2016-05-29T18:55:00.000000064


In [8]:
rainfall_map = ds.rainfall_amount.sum(dim='time').hvplot.quadmesh(
    x='longitudes', 
    y='latitudes',
    frame_width=500, 
    rasterize=True,
    tiles='OSM', 
    project=True, 
    geo=True, 
    clim=(0.1, 100), 
    cmap='rainbow', 
    clabel='rainfall amount (mm)')

rainfall_map.opts('Image', clipping_colors={'min': 'transparent', 'NaN': 'gray'}, alpha=0.5, toolbar='above')

rainfall_map * braunsbach.hvplot.points(x='x', y='y', geo=True, color='black')

:DynamicMap   []
   :Overlay
      .Tiles.I  :Tiles   [x,y]
      .Image.I  :Image   [longitudes,latitudes]   (rainfall_amount)
      .Points.I :Points   [x,y]

# [WIP] Rainfall map with time slider and marker for Braunsbach

Unfortunately the addition of `hvplot.points` to the map results in resizing the plot when the time slider is dragged. I did not yet find out how to solve this...

In [9]:
rainfall_map = ds.rainfall_amount.hvplot.quadmesh(
    x='longitudes', 
    y='latitudes',
    frame_width=500, 
    rasterize=True,
    tiles='OSM', 
    project=True, 
    geo=True, 
    clim=(0.1, 10), 
    cmap='rainbow', 
    clabel='rainfall amount (mm)')

rainfall_map.opts('Image', clipping_colors={'min': 'transparent', 'NaN': 'gray'}, alpha=0.5, toolbar='above')

rainfall_map * braunsbach.hvplot.points(x='x', y='y', geo=True, project=True, color='black')

:DynamicMap   [time]
   :Overlay
      .Tiles.I  :Tiles   [x,y]
      .Image.I  :Image   [longitudes,latitudes]   (rainfall_amount)
      .Points.I :Points   [x,y]